In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from random import choice
import os
import scipy as sp
import scipy.fftpack
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter


In [10]:
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\global_hypha_infos.pick"
global_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_hypha_infos.pick"
time_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\global_plate_infos.pick"
global_plate_info = pd.read_pickle(path_save)

def get_branch(row):
    if row['Plate']==94:
        hyph = row['end']
        name = f'branches/ts_{hyph}_auto.npy'
        if not os.path.isfile(name):
            return(None)
        else:
            t_real = np.load(name)
            return(row['time_since_emergence'] in t_real)
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]

global_hypha_info['lapse']=global_hypha_info['time_stop_growth']-global_hypha_info['time_init_growth']
time_hypha_info['distance_final_pos_smooth'] = time_hypha_info['distance_final_pos'].rolling(window = 5).mean()
time_hypha_info['new_branch'] = time_hypha_info.apply(get_branch, axis=1)

In [28]:
global_hypha_info['hypha_unique'] = global_hypha_info['hypha'].astype(str)+'_'+global_hypha_info['Plate'].astype(str)
time_hypha_info['hypha_unique'] = time_hypha_info['end'].astype(str)+'_'+time_hypha_info['Plate'].astype(str)


plate = 792
roundin = 1000
plates = [792]

select_global = global_hypha_info.loc[global_hypha_info['Plate'].isin(plates)]

select_global = select_global.loc[select_global['lapse']>10]
hyphae_interest= select_global['hypha_unique'].unique()
last_time = np.max(select_time_plate.loc[select_time_plate['out_study']==0]['time_since_begin'])

select_time = time_hypha_info.loc[time_hypha_info['Plate'].isin(plates)]

select_time = select_time.loc[select_time['hypha_unique'].isin(hyphae_interest)]

select_time = select_time.loc[select_time['speed'].between(50,400)]
select_time['speed'] = select_time['speed'].astype(float)
select_time = select_time.loc[select_time['distance_final_pos']>400]
select_time = select_time.loc[select_time['in_ROI']=='True']
hyphae = select_time.groupby("hypha_unique")["timestep"].count()>=15
select_time = select_time.set_index("hypha_unique")
select_time = select_time.loc[hyphae[select_time.index]]
select_time['time_since_begin_exp'] = roundin*select_time['time_since_begin_exp'].astype(int)/roundin 
aggregate = 8
select_time['time_since_begin_exp_round'] = (select_time['time_since_begin_exp'].astype(int))//aggregate*aggregate
select_time = select_time.loc[select_time['time_since_begin_exp']<=last_time]

hyphae_interest = select_time['end'].unique()

In [13]:
select_time['time_since_begin_exp_round']

hypha_unique
0_792          0
1_792          0
7_792          0
8_792          0
0_792          0
            ... 
68480_792    216
68480_792    216
68480_792    216
68480_792    222
68480_792    222
Name: time_since_begin_exp_round, Length: 1124, dtype: int32

In [29]:
import joypy
from matplotlib import cm

fig, axes = joypy.joyplot(select_time,column="speed",by ='time_since_begin_exp_round' ,colormap=cm.autumn_r,title=plate,fade=True,figsize=(6,5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
plates = time_hypha_info['Plate'].unique()
time_hypha_table = {}
for plate in plates: 
    time_hypha_table[plate]=global_hypha_info.loc[global_hypha_info['Plate']==plate].copy()
    select_time_plate = time_plate_info.loc[time_plate_info['Plate']==plate]
    last_time = np.max(select_time_plate.loc[select_time_plate['out_study']==0]['time_since_begin'])
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
    select_global = global_hypha_info.loc[global_hypha_info['Plate']==plate]
    select_global = select_global.loc[select_global['lapse']>40]
#     select_global = select_global.loc[select_global['mean_speed']>60]
    hyphae_interest= select_global['hypha'].unique()
    select_time = select_time.loc[select_time['end'].isin(hyphae_interest)]
    select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
    select_time = select_time.loc[select_time['speed'].between(40,400)]
    select_time = select_time.loc[select_time['distance_final_pos']>100]
    select_time = select_time.loc[select_time['in_ROI']=='True']
    select_time = select_time.loc[select_time['time_since_begin_exp']<=last_time]
    select_time['speed'] = select_time['speed'].astype(float)

In [5]:
select_time.plot('time_since_begin_exp','speed',kind='scatter')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time_since_begin_exp', ylabel='speed'>

In [15]:
select_time.plot.hexbin(x="speed", y="time_since_begin_exp_round", gridsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='speed', ylabel='time_since_begin_exp_round'>

In [81]:
import joypy
from matplotlib import cm

fig, axes = joypy.joyplot(select_time,column="speed",by ='time_since_begin_exp_round' ,ylabels=False, xlabels=False, 
                          grid=False, fill=False, background='k', linecolor="w", linewidth=1,
                          legend=False, overlap=0.5, figsize=(6,5),kind="counts", bins=20)
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
for a in axes[:-1]:
    a.set_xlim([-200,600])   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
!pip install joypy

In [33]:
df = pd.concat([select_time.groupby('time_since_begin_exp')['speed'].mean()], axis=1).copy()
df = df.sort_index()
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(df.index,df['speed'],label=plate)
ax.set

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<bound method Artist.set of <AxesSubplot:>>

In [30]:
df['speed']

time_since_begin_exp
0.000000     231.290591
1.116667     246.775005
2.133333     242.173418
3.133333     216.080811
4.133333     208.793838
                ...    
76.266667    291.013586
77.266667    268.545814
78.266667    270.217940
79.266667    272.571430
80.266667    262.784631
Name: speed, Length: 81, dtype: float64

In [11]:
select_time

,end,distance_final_pos,timedelta,time_since_emergence,speed,timestep,timestep_init,time_init,degree,width_tip_edge,...,time_since_begin_exp,width_root_edge,Plate,folder,t,density_window500,density_window1000,density_window2000,distance_final_pos_smooth,new_branch
1,1,15976.8,1.11667,0,198.007,0,0,0,1,8.57501,...,0,NaN,94,Analysis_1636046622780809853_0_100_Version9,0,1739.13,NaN,NaN,6281.170265,False
19,19,17232.6,1.11667,0,296.98,0,0,0,1,7.58277,...,0,NaN,94,Analysis_1636046622780809853_0_100_Version9,0,2374.49,NaN,NaN,3984.035661,False
56,56,2062.23,1.11667,0,159.966,0,0,0,1,None,...,0,NaN,94,Analysis_1636046622780809853_0_100_Version9,0,3123.48,NaN,NaN,671.261785,False
139,139,15770.2,1.11667,0,270.209,0,0,0,1,8.93035,...,0,NaN,94,Analysis_1636046622780809853_0_100_Version9,0,1166.38,NaN,NaN,3216.089259,False
1,1,15852.6,1.01667,1.11667,192.039,1,0,0,1,8.50418,...,1.11667,NaN,94,Analysis_1636046622780809853_0_100_Version9,1,1704.35,NaN,NaN,6209.875340,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18054,18054,1948.69,1,37.1167,213.224,80,43,43.15,1,8.4672,...,80.2667,NaN,94,Analysis_1636046622780809853_0_100_Version9,80,2434.78,NaN,NaN,395.750615,False
18897,18897,3333.32,1,36.1167,288.245,80,44,44.15,1,5.48694,...,80.2667,NaN,94,Analysis_1636046622780809853_0_100_Version9,80,1477.1,NaN,NaN,675.097396,False
24489,24489,3371.07,1,30.1167,292.301,80,50,50.15,1,9.54535,...,80.2667,NaN,94,Analysis_1636046622780809853_0_100_Version9,80,1168.7,NaN,NaN,754.761693,False
31802,31802,3035.19,1,24.1167,280.05,80,56,56.15,1,10.4116,...,80.2667,NaN,94,Analysis_1636046622780809853_0_100_Version9,80,1161.74,NaN,NaN,732.984782,False
